In [1]:
import sys
import glob
sys.path.append('./extract_patches.py')
import extract_patches as ep

In [ ]:
file_path = 'I:/TCGA_bladder/TCGA_bladder/0b804fd8-70c4-458d-a172-3329118ca6d3/'
svs_path = r.glob(file_path)